In [1]:
import numpy as np
from numba import njit, prange, float64
import exafmm.laplace as laplace
import time

In [5]:
@njit('(float64[:,:], float64[:,:], float64[:,:],float64, float64)', cache=True, fastmath=True, parallel=True)
def compute_acc_poisson(pos,mass,charge, k, softening):
    """ Computes the Acceleration of N bodies
	Args:
		pos (type=np.array, size= Nx3): x, y, z positions of the N particles
		mass (type=np.array, size= Nx1): mass of the particles
        k (float): Coulomb constant
		softening (float): softening parameter

	Returns:
		acc (type=np.array, size= Nx3): ax, ay, az accelerations of the N particles
	"""
    n = pos.shape[0]

    # Copy the array view so for the next loop to be faster
    x = pos[:,0].copy()
    y = pos[:,1].copy()
    z = pos[:,2].copy()

    # Ensure mass is a contiguous 1D array (cheap operation)
    assert mass.shape[1] == 1
    contig_mass = mass[:,0].copy()
    
    # Ensure charge is a contiguous 1D array (cheap operation)
    assert charge.shape[1] == 1
    contig_charge = charge[:,0].copy()

    acc = np.empty((n, 3), pos.dtype)

    for i in prange(n):
        ax, ay, az = 0.0, 0.0, 0.0

        for j in range(n):
            dx = x[i] - x[j]  
            dy = y[i] - y[j]
            dz = z[i] - z[j]
            tmp = (dx**2 + dy**2 + dz**2 + softening**2)
            factor = contig_charge[j] / (tmp * np.sqrt(tmp))
            ax += dx * factor
            ay += dy * factor
            az += dz * factor

        acc[i, 0] = k * contig_charge[i]/contig_mass[i] * ax
        acc[i, 1] = k * contig_charge[i]/contig_mass[i] * ay
        acc[i, 2] = k * contig_charge[i]/contig_mass[i] * az

    return acc

def FMM_acc_poisson(positions, masses, charges,P,nbpl):
    """
    Compute the accelerations of a collection of particles interacting electrostatically.

    Parameters:
    - positions (List[Tuple[float, float, float]]): A list of tuples, where each tuple represents the x, y, z coordinates of a particle's position.  
    - masses (List[float]): A list of mass values for each particle. 
    - charges (List[float]): A list of charge values for each particle.
    - P: order of expansion of the FMM
    - nbpl: max number of bodies per leaf
    Returns:
    - accelerations: A numpy array of shape (n, 3) representing the acceleration of each particle in the x, y, z directions.
    """
    
    # Ensure that the inputs are numpy arrays
    #positions = np.array(positions)
    #masses = np.array(masses)
    #charges = np.array(charges)
    

    # Number of particles
    n = len(positions)

    # create a list of source instances
    sources = laplace.init_sources(positions, charges)
    # create a list of target instances
    targets = laplace.init_targets(positions)

    # create a LaplaceFmm instance
    fmm = laplace.LaplaceFmm(p=P, ncrit=nbpl, filename="test_file.dat")

    # setup the tree
    tree = laplace.setup(sources, targets, fmm)

    # evaluate potential and its gradient
    trg_values = laplace.evaluate(tree, fmm)

    # compute forces on each particle
    k=8.9875517923*1e9 # Coulomb constant
    forces = -k*charges[:, np.newaxis] * trg_values[:, 1:]  # Multiply charges by electric field

    # compute accelerations of each particle
    accelerations = forces / masses[:, np.newaxis]
     
    return accelerations

In [6]:
softening=1e-6
k=8.9875517923*1e9

P=2
nbpl=100

In [7]:
N = 1000000
pos = np.random.uniform(-100, 100, size=(N, 3))
mass = np.random.uniform(0, 10, size=(N, 1))
charge = np.random.uniform(-1, 1, size=(N, 1))

In [42]:
%%timeit -r 3 -n 1
compute_acc_poisson(pos,mass,charge, k, softening)

28.3 ms ± 78.3 µs per loop (mean ± std. dev. of 10 runs, 1,000 loops each)


In [8]:
%%timeit -r 3 -n 1
FMM_acc_poisson(pos, mass.ravel(), charge.ravel(),P,nbpl)

1.15 s ± 281 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
